In [1]:
import numpy as np
import suite2p
from scanimagetiffio import SITiffIO
#import utils_image from the parent folder
import sys
sys.path.append("..")
from utils_image import UnrotateCropFrame, RegFrame, compute_zpos_sp


In [2]:
#load the zstack tiff and calculate the mean frames for each stack

zstacktiff = '/home/zilong/Desktop/2D2P/Data/162/162_zstack_19062023__00002.tif'
ztstackrelog = '/home/zilong/Desktop/2D2P/Data/162/REdata_20230619_165713.txt'

S = SITiffIO()
S.open_tiff_file(zstacktiff, "r")
S.open_rotary_file(ztstackrelog)
S.interp_times()  # might take a while...

#get all frames and angles in S
Array = []; 
for i in range(S.get_n_frames()):
    Array.append(S.get_frame(i+1))
#change Array to a numpy array
Array = np.array(Array)
Angle = S.get_all_theta()

# unrotate each frame in the tiff file with the detected rotation center
unrotcropFrames = UnrotateCropFrame(Array, Angle, rotCenter=[256,256])


num_v=5; num_s=41; num_f=10
# reshape the unrotFrames to a 5D array
unrotcropFrames = unrotcropFrames.reshape(
    num_v, num_s, num_f, unrotcropFrames.shape[1], unrotcropFrames.shape[2]
)
# average across the first and third dimension
meanZstacks = np.mean(unrotcropFrames, axis=(0, 2))
             

Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/162/REdata_20230619_165713.txt
Finished calculating rotations and times.
The rotary encoder file has 22266 timestamps in it.
Counted 2050 frames


In [3]:
#load the imaging tiff 
imagingtiff = '/home/zilong/Desktop/2D2P/Data/162/162_block1_19062023__00001.tif'
imagingrelog = '/home/zilong/Desktop/2D2P/Data/162/REdata_20230619_170256.txt'

S = SITiffIO()
S.open_tiff_file(imagingtiff, "r")
S.open_rotary_file(imagingrelog)
#extract the last self.numFrames frames from the tiff file
tailArray, tailAng = S.tail(1000)
        
# unrotate each frame in the tiff file with the detected rotation center
unrotFrames  = UnrotateCropFrame(tailArray, tailAng, rotCenter=[256,256])

#perform image registraion
meanRegImg, regFrames = RegFrame(unrotFrames)

Loading rotary encoder file: /home/zilong/Desktop/2D2P/Data/162/REdata_20230619_170256.txt
Calculating rotations and times from rotary encoder data...
Finished calculating rotations and times.
The rotary encoder file has 79743 timestamps in it.
Counted 18000 frames
Reference frame, 8.15 sec.
Registered 200/1000 in 2.29s
Registered 400/1000 in 5.40s
Registered 600/1000 in 7.46s
Registered 800/1000 in 9.34s
Registered 1000/1000 in 11.22s


In [4]:
#comnpute the z drift of each frame
ops = suite2p.default_ops()
zcorr = compute_zpos_sp(meanZstacks, regFrames, ops)

1 planes, 0/1000 frames, 0.66 sec.
11 planes, 0/1000 frames, 3.29 sec.
21 planes, 0/1000 frames, 5.78 sec.
31 planes, 0/1000 frames, 8.09 sec.
40 planes, 0/1000 frames, 10.16 sec.
1 planes, 100/1000 frames, 10.58 sec.
11 planes, 100/1000 frames, 13.02 sec.
21 planes, 100/1000 frames, 15.40 sec.
31 planes, 100/1000 frames, 17.76 sec.
40 planes, 100/1000 frames, 19.83 sec.
1 planes, 200/1000 frames, 20.26 sec.
11 planes, 200/1000 frames, 22.58 sec.
21 planes, 200/1000 frames, 25.01 sec.
31 planes, 200/1000 frames, 27.26 sec.
40 planes, 200/1000 frames, 29.39 sec.
1 planes, 300/1000 frames, 29.82 sec.
11 planes, 300/1000 frames, 32.24 sec.
21 planes, 300/1000 frames, 34.50 sec.
31 planes, 300/1000 frames, 36.84 sec.
40 planes, 300/1000 frames, 38.93 sec.
1 planes, 400/1000 frames, 39.36 sec.
11 planes, 400/1000 frames, 41.80 sec.
21 planes, 400/1000 frames, 44.14 sec.
31 planes, 400/1000 frames, 46.36 sec.
40 planes, 400/1000 frames, 48.46 sec.
1 planes, 500/1000 frames, 49.00 sec.
11 pla

In [6]:
#imshow zcorr with auto aspect ratio
fig = plt.figure(figsize=(512/100,512/100),dpi=100)
plt.imshow(zcorr, cmap='gray', aspect='auto')
plt.axis('off')
plt.show()

NameError: name 'plt' is not defined